In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# G1

In [ ]:
import numpy as np

def objective_function(x):
    x = np.array(x).reshape(-1, 13)
    f = 5 * np.sum(x[:, :4], axis=1) - 5 * np.sum(x[:, :4] ** 2, axis=1) - np.sum(x[:, 4:], axis=1)
    return f

def constraint_function(x):
    x = np.array(x).reshape(-1, 13)
    g1 = 2 * x[:, 0] + 2 * x[:, 1] + x[:, 9] + x[:, 10] - 10
    g2 = 2 * x[:, 0] + 2 * x[:, 2] + x[:, 9] + x[:, 11] - 10
    g3 = 2 * x[:, 1] + 2 * x[:, 2] + x[:, 10] + x[:, 11] - 10
    g4 = -8 * x[:, 0] + x[:, 9]
    g5 = -8 * x[:, 1] + x[:, 10]
    g6 = -8 * x[:, 2] + x[:, 11]
    g7 = -2 * x[:, 3] - x[:, 4] + x[:, 9]
    g8 = -2 * x[:, 5] - x[:, 6] + x[:, 10]
    g9 = -2 * x[:, 7] - x[:, 8] + x[:, 11]

    return np.vstack((g1, g2, g3, g4, g5, g6, g7, g8, g9))  

def constraints(x):

    return np.all(constraint_function(x) <= 0, axis=0)

def objective_function_with_constraints(x):
    if not constraints(x):
        return 0
    else:
        return objective_function(x)


In [ ]:
def get_real_pop(): # populacao com todas as particulas válidas 
  filename = '/content/populacao_teste.txt'

  # Lista para armazenar os indivíduos lidos
  real_population = []

  # Ler o arquivo de texto
  with open(filename, 'r') as file:
      for line in file:
          individual = list(map(float, line.strip().split()))
          real_population.append(individual)
  real_population  = np.array(real_population)
  return real_population

## pso 

In [ ]:
import numpy as np
import pandas as pd

# Número de partículas
n_particles = 50

# Número de dimensões
n_dimensions = 13

# Limites inferiores e superiores para cada dimensão
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]

# Parâmetros do PSO
c1 = 2
c2 = 2
w = 0.7
max_iter = 300
n_experiments = 30

# Armazenar resultados
results = []

pso_final = []
#PSO
for exp in range(n_experiments):
    # Inicializar enxame
    #particles = np.empty((n_particles, n_dimensions))
    #for i in range(n_particles):  ## gerar individuos válidos 
    #    while True:
    #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
    #        if objective_function_with_constraints(particle) != 0:
    #            particles[i] = particle
    #            break
    particles = get_real_pop()
    personal_best_scores = np.array([objective_function_with_constraints(p) for p in particles])

    velocities = np.zeros((n_particles, n_dimensions)) #inicializar velocidades com 0 
    personal_best_positions = particles.copy()

    # Inicializar o melhor global
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = min(personal_best_scores)

    # Loop principal
    for i in range(max_iter):
        print(f'------------------')
        print(f'experimento {exp+1} de {n_experiments}')
        print(f'iteração {i+1}')
        print(f'Global best score: {global_best_score}')
        for j in range(n_particles):
            # Atualizar velocidade
            velocities[j] = (w * velocities[j] +
                             c1 * np.random.rand() * (personal_best_positions[j] - particles[j]) +
                             c2 * np.random.rand() * (global_best_position - particles[j]))
            
            # Atualizar posição
            particles[j] += velocities[j]
            
            # Aplicar limites
            particles[j] = np.clip(particles[j], lb, ub)
            
            # Atualizar o melhor pessoal
            score = objective_function_with_constraints(particles[j])
            if score < personal_best_scores[j]:
                personal_best_positions[j] = particles[j].copy()
                personal_best_scores[j] = score
                
                # Atualizar o melhor global
                if score < global_best_score:
                    global_best_position = particles[j].copy()
                    global_best_score = score

        # Armazenar resultados
        results.append({
            'Experiment': exp+1,
            'Iteration': i+1,
            'Best Fitness': np.min(global_best_score),
            'Best Particle': global_best_position,
            'Worst Fitness': np.max(personal_best_scores),
            'Average Fitness': np.mean(personal_best_scores),
            'Standard Deviation': np.std(personal_best_scores),
            'Median Fitness': np.median(personal_best_scores),
        })
    pso_final.append(np.min(global_best_score))
        

# Criar dataframe com os resultados
df = pd.DataFrame(results)

df.head(5)



In [ ]:
df.head(1000)

In [ ]:
df_pso = df.sort_values('Best Fitness', ascending=True).reset_index()
best_PSO_individual = df_pso.loc[0, 'Best Particle']
fitness_best_PSO_individual = df_pso.loc[0, 'Best Fitness']
df_pso.head()

In [ ]:
best_PSO_individual


## ga

In [ ]:
import math
import random
import numpy as np
import pandas as pd

# Definindo os parâmetros do algoritmo genético
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
crossover_prob = 0.9
mutation_prob = 0.12
n_experiments = 30

results_ga = []
ga_final = []

for exp in range(n_experiments):
   # Inicialização da população
   # Inicializar enxame
   #particles = np.empty((n_particles, n_dimensions))
   #for i in range(n_particles):  ## gerar individuos válidos 
   #    while True:
   #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
   #        if objective_function_with_constraints(particle) != 0:
   #            particles[i] = particle
   #            break
   #population = particles
    population = get_real_pop()
    for gen in range(generations):
        # Avaliação da população
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        print(f'------------------')
        print(f'experimento {exp+1} de {n_experiments}')
        print(f'generation {gen+1}')
        print(f'Global best score: {min(fitness_values)}')
        print(f'Global worst score: {max(fitness_values)}')

        # Seleção por torneio
        parents = []
        while len(parents) < len(population):
            contenders_indices = random.sample(range(len(population)), 2)
            contenders = [population[i] for i in contenders_indices]
            contenders_fitness = [fitness_values[i] for i in contenders_indices]
            best_contender = contenders[contenders_fitness.index(min(contenders_fitness))]
            parents.append(best_contender)

        # Garantindo que o número de pais seja par
        if len(parents) % 2 != 0:
            parents.pop()



        # Assegurando que o melhor indivíduo sempre estará na próxima geração
        best_individual = population[np.argmin(fitness_values)]

        # Crossover Uniforme
        offspring = []
        for i in range(0, len(parents), 2):
            parent1 = parents[i]
            parent2 = parents[i+1]
            child1 = parent1.copy()
            child2 = parent2.copy()
            for j in range(len(parent1)):
                if random.random() < crossover_prob:
                    child1[j] = parent2[j]
                    child2[j] = parent1[j]
            offspring.extend([child1, child2])
            
       # Crossover Blend
        alpha = 0.5
        for i in range(0, len(parents), 2):
            parent1 = np.array(parents[i])
            parent2 = np.array(parents[i+1])

            d = np.abs(parent1 - parent2)
            low = np.clip(np.minimum(parent1, parent2) - alpha * d, lb, ub)
            high = np.clip(np.maximum(parent1, parent2) + alpha * d, lb, ub)
            child1 = np.random.uniform(low, high)
            child2 = np.random.uniform(low, high)

            offspring.extend([child1.tolist(), child2.tolist()])
            

            


        # Substituição na população
        population = offspring + [best_individual]

        # Manter o tamanho da população constante em 50 indivíduos
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        pop_with_fitness = list(zip(population, fitness_values))
        pop_with_fitness.sort(key=lambda x: x[1])
        population, fitness_values = map(list, zip(*pop_with_fitness[:50]))


        # Mutação
        best_index = np.argmin(fitness_values)  
        for i in range(len(population)):
            if i != best_index and np.random.rand() <= mutation_prob:
                mutation_point = np.random.randint(genes)
                new_value = np.random.uniform(lb[mutation_point], ub[mutation_point])
                temp_individual = population[i].copy()
                temp_individual[mutation_point] = new_value
                if constraints(temp_individual):
                    population[i] = temp_individual

        # Reavaliação da população após crossover e mutação
        fitness_values = [objective_function_with_constraints(ind) for ind in population]

        best_fitness = min(fitness_values)
        worst_fitness = max(fitness_values)
        mean_fitness = np.mean(fitness_values)
        std_dev_fitness = np.std(fitness_values)

        results_ga.append({
            'experimento':exp+1,
            'geração': gen,
            'melhor_fitness': best_fitness,
            'melhor_individuo': population[np.argmin(fitness_values)],
            'pior_fitness': worst_fitness,
            'media_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'numero_individuos': len(population),
        })

         
    ga_final.append(best_fitness)   

df_ga = pd.DataFrame(results_ga)
df_ga.head(5)




In [ ]:
df_ga.head(1000)

In [ ]:
df_ga = df_ga.sort_values('melhor_fitness', ascending=True).reset_index()
best_ga_individual = df_ga.loc[0, 'melhor_individuo']
fitness_best_ga_individual = df_ga.loc[0, 'melhor_fitness']
df_ga.head()

## de

In [ ]:
# Definindo os parâmetros do de
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
crossover_prob = 0.6
mutation_prob = 0.1
F = 0.8  # Fator de escala para a mutação

results_de = []
de_final = []

for exp in range(30):
   # Inicialização da população
   # Inicializar enxame
   #particles = np.empty((n_particles, n_dimensions))
   #for i in range(n_particles):  ## gerar individuos válidos 
   #    while True:
   #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
   #        if objective_function_with_constraints(particle) != 0:
   #            particles[i] = particle
   #            break
   #population = particles
    population = get_real_pop()

    # Loop das gerações
    for gen in range(generations):
        # Avaliação da população
        fitness_values = [objective_function_with_constraints(ind) for ind in population]

        # Salvar o melhor indivíduo para elitismo
        best_index = np.argmin(fitness_values)
        best_individual = population[best_index].copy()

        # Evolução Diferencial
        new_population = population.copy()
        for i in range(pop_size):
            indices = [idx for idx in range(pop_size) if idx != i]
            a, b, c = population[np.random.choice(indices, 3, replace=False)]
            mutant = np.clip(a + F * (b - c), lb, ub)  # Mutação
            cross_points = np.random.rand(genes) < crossover_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, genes)] = True
            trial = np.where(cross_points, mutant, population[i])  # Crossover
            if objective_function_with_constraints(trial) < fitness_values[i]:  # Aceitação
                new_population[i] = trial

        population = new_population

        # Elitismo: garantir que o melhor indivíduo esteja na nova população
        new_fitness_values = [objective_function_with_constraints(ind) for ind in population]
        worst_index = np.argmax(new_fitness_values)
        if new_fitness_values[worst_index] > fitness_values[best_index]:
            population[worst_index] = best_individual

        # Estatísticas da geração
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        best_fitness = min(fitness_values)
        worst_fitness = max(fitness_values)
        mean_fitness = np.mean(fitness_values)
        std_dev_fitness = np.std(fitness_values)

        results_de.append({
            'experimento': exp+1,
            'geracao': gen,
            'melhor_fitness': best_fitness,
            'melhor_particula': population[np.argmin(fitness_values)],
            'pior_fitness': worst_fitness,
            'media_dos_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'n_de_individuos': len(population),
        })
        print(f'Experimento {exp+1}, Geração {gen+1}')
        print(f'Melhor partícula: {population[np.argmin(fitness_values)]}, Fitness: {best_fitness}')
        print(f'Pior partícula: {population[np.argmax(fitness_values)]}, Fitness: {worst_fitness}')
        print('---'*20)
    de_final.append(best_fitness)

df_de = pd.DataFrame(results_de)
df_de.head(5)


In [ ]:
df_de.head(1000)

In [ ]:
df_de = df_de.sort_values('melhor_fitness', ascending=True).reset_index()
best_de_individual = df_de.loc[0, 'melhor_particula']
fitness_best_de_individual = df_de.loc[0, 'melhor_fitness']
df_de.head()

## ES

#parametros globais + EE-CMA

In [ ]:
import numpy as np
from scipy.linalg import sqrtm

# Definindo os parâmetros da evolução estratégica
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
sigma = 0.1  # Desvio padrão para a mutação
tau = 1/np.sqrt(2*genes)  # Parâmetro de autoadaptação

results_es = []
es_g_final = []



# Loop dos experimentos
for exp in range(30):
  population = get_real_pop()
  sigmas = sigma * np.ones((pop_size, genes, genes))
  # Loop das gerações
  # Loop das gerações
  for gen in range(generations):
      # Avaliação da população
      fitness_values = [objective_function_with_constraints(ind) for ind in population]

      # Elitismo: guardar o melhor indivíduo
      best_index = np.argmin(fitness_values)
      best_individual = population[best_index].copy()

      # Estratégia Evolutiva
      for i in range(pop_size):
          # Seleção de pais aleatórios
          parent_indices = np.random.choice(pop_size, 2, replace=False)
          parents = population[parent_indices]

          # CMA-ES
          mean = np.mean(parents, axis=0)
          cov = np.cov(parents.T)
          child = np.random.multivariate_normal(mean, cov)
          child = np.clip(child, lb, ub)

          # Adição de mutação
          child += sigma * np.random.randn(genes)
          child = np.clip(child, lb, ub)

          # Seleção (mu, sigma)
          if objective_function_with_constraints(child) < fitness_values[i]:
              population[i] = child

      # Elitismo: substituir o pior indivíduo pelo melhor da geração anterior, se necessário
      fitness_values = [objective_function_with_constraints(ind) for ind in population]
      worst_index = np.argmax(fitness_values)
      if fitness_values[worst_index] > objective_function_with_constraints(best_individual):
          population[worst_index] = best_individual

      # Estatísticas da geração
      best_fitness = min(fitness_values)
      worst_fitness = max(fitness_values)
      mean_fitness = np.mean(fitness_values)
      std_dev_fitness = np.std(fitness_values)

      results_es.append({
          'experimento': exp+1,
          'geracao': gen,
          'melhor_fitness': best_fitness,
          'melhor_particula': population[np.argmin(fitness_values)],
          'pior_fitness': worst_fitness,
          'media_dos_fitness': mean_fitness,
          'desvio_padrao': std_dev_fitness,
          'n_de_individuos': len(population),
      })

      # Imprime as melhores e piores partículas
      print(f'Experimento {exp+1}, Geração {gen+1}')
      print(f'Melhor partícula: {population[np.argmin(fitness_values)]}, Fitness: {best_fitness}')
      print(f'Pior partícula: {population[np.argmax(fitness_values)]}, Fitness: {worst_fitness}')
      print('---'*20)
      
  es_g_final.append(best_fitness)

df_g_es = pd.DataFrame(results_es)
df_g_es.head(5)





In [ ]:
df_g_es.head(1000)

In [ ]:
df_g_es = df_g_es.sort_values('melhor_fitness', ascending=True).reset_index()
best_g_es_individual = df_g_es.loc[0, 'melhor_particula']
fitness_best_g_es_individual = df_g_es.loc[0, 'melhor_fitness']
df_g_es.head()

## resultados obtidos

In [ ]:
resultados_G1 = pd.DataFrame({
    'EVO. Alg.':['PSO','GA','DE','ES'],
    'Best Fitness':[fitness_best_PSO_individual,fitness_best_ga_individual,fitness_best_de_individual,fitness_best_g_es_individual],
    'Best Particle EA':[best_PSO_individual,best_ga_individual,best_de_individual,best_g_es_individual]})

resultados_G1 = resultados_G1.sort_values('Best Fitness',ascending=True).reset_index()
resultados_G1.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


data = pd.DataFrame({
    'PSO': pso_final,
    'GA': ga_final,
    'DE': de_final,
    'ES G': es_g_final,
})


plt.figure(figsize=(10, 6))
sns.boxplot(data=data)
plt.title('Boxplot dos Fitness Finais')
plt.ylabel('Fitness')
plt.show()


# G2

## ES

## resultados obtidos